In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from binance.client import Client
from matplotlib.backends.backend_pdf import PdfPages

class CryptoDataAnalyzer:
    def __init__(self, api_key, api_secret):
        # Initialize the Binance API client
        self.client = Client(api_key, api_secret)
        
        # Define the cryptocurrency symbols and look-back periods
        self.symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'ICPUSDT', 'ADAUSDT', 'LTCUSDT']
        self.look_back_periods = [3, 7, 35]
        self.interval = '1h'
        self.data_frames = {}
        self.correlation_matrices = {}
        self.pdf_filename = "correlation_matrices.pdf"

    def fetch_historical_data(self):
        """
        Fetch historical price data for specified symbols and periods.
        """
        for symbol in self.symbols:
            symbol_data = {}
            for period in self.look_back_periods:
                # Retrieve historical klines data from Binance
                klines = self.client.get_historical_klines(symbol, self.interval, f"{period} days ago UTC")
                
                # Process and store klines data as a DataFrame
                df = self.process_klines_data(klines)
                symbol_data[period] = df
            self.data_frames[symbol] = symbol_data

    def process_klines_data(self, klines):
        """
        Process raw klines data and convert it to a DataFrame.
        """
        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                           'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume',
                                           'taker_buy_quote_asset_volume', 'ignore'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df['close'] = pd.to_numeric(df['close'])
        return df

    def calculate_correlation_matrices(self):
        """
        Calculate correlation matrices for different look-back periods.
        """
        for period in self.look_back_periods:
            correlation_matrix = np.zeros((len(self.symbols), len(self.symbols)))
            for i, symbol1 in enumerate(self.symbols):
                for j, symbol2 in enumerate(self.symbols):
                    # Calculate the correlation coefficient
                    correlation = self.calculate_correlation(symbol1, symbol2, period)
                    correlation_matrix[i, j] = correlation if not np.isnan(correlation) else 0.0
            self.correlation_matrices[period] = correlation_matrix

    def calculate_correlation(self, symbol1, symbol2, period):
        """
        Calculate the correlation coefficient between two symbols for a given period.
        """
        return np.corrcoef(self.data_frames[symbol1][period]['close'], self.data_frames[symbol2][period]['close'])[0, 1]

    def plot_and_save_correlation_matrices(self):
        """
        Plot correlation matrices and save them to a PDF file.
        """
        pdf_pages = PdfPages(self.pdf_filename)
        for period, correlation_matrix in self.correlation_matrices.items():
            plt.figure(figsize=(9, 7))
            sns.heatmap(correlation_matrix, annot=True, cmap='RdYlGn', linewidths=0, vmin=-1, vmax=1,
                        square=True, cbar=False, fmt=".3f")
            plt.title(f"Correlation Matrix {period}-Day Look-back Period")
            plt.xticks(ticks=np.arange(len(self.symbols)) + 0.5, labels=self.symbols, rotation=45)
            plt.yticks(ticks=np.arange(len(self.symbols)) + 0.5, labels=self.symbols, rotation=0)
            plt.tight_layout()
            plt.savefig(pdf_pages, format='pdf')
            plt.close()
        pdf_pages.close()

def main():
    # Replace with your Binance API key and secret
    api_key = 'YOUR_API_KEY'
    api_secret = 'YOUR_API_SECRET'

    analyzer = CryptoDataAnalyzer(api_key, api_secret)
    analyzer.fetch_historical_data()
    analyzer.calculate_correlation_matrices()
    analyzer.plot_and_save_correlation_matrices()

if __name__ == "__main__":
    main()

